## Get the Metadata for the 990 IRS files

In [3]:
from pymongo import MongoClient

#import our own libs
# METHOD 1
#import import_ipynb
#import IRS990libs
# METHOD 2 - This will reaload without restarting notebook (debugging)
%run IRS990libs.ipynb

In [4]:
year = 2017

In [5]:
# SET TO TRUE IF YOU WANT TO CLEAN DATABASE
DUMP_DB = True

uri = "mongodb://mongo/tweets"
client = MongoClient(uri)

print( 'List of databases in MongoDB:' )
print(client.list_database_names())

# database
db = client['irs990']

# collection
dbYear = db['%s' % year]

# if we already have documents then drop them and start
# clean on this collection
if DUMP_DB and dbYear.count_documents({}) > 0:
    print( 'Drop database and start over with archive')
    dbYear.drop()

List of databases in MongoDB:
['admin', 'config', 'local']
List of databases in MongoDB:
['admin', 'config', 'local']
List of databases in MongoDB:
['admin', 'config', 'local']
List of databases in MongoDB:
['admin', 'config', 'local']
List of databases in MongoDB:
['admin', 'config', 'local']
List of databases in MongoDB:
['admin', 'config', 'local']
List of databases in MongoDB:
['admin', 'config', 'local']
List of databases in MongoDB:
['admin', 'config', 'local']


In [6]:
%run IRS990libs.ipynb

documents = classy990forms(config='validElements.conf', debug=False)
documents.year( year )
documents

IRS 990 form for year [2017]: 489013 documents

IRS 990 form for year [2017]: 489013 documents

IRS 990 form for year [2017]: 489013 documents

IRS 990 form for year [2017]: 489013 documents

IRS 990 form for year [2017]: 489013 documents

IRS 990 form for year [2017]: 489013 documents

IRS 990 form for year [2017]: 489013 documents

IRS 990 form for year [2017]: 489013 documents

In [7]:
for x in documents:
    
    index = "%s_%s" % ( x['EIN'], year)
    
    sys.stdout.write('Dowload EIN:%s\r' % index)
    sys.stdout.flush() 
    
    dbYear.update_one( {"_id" : index}, {"$set":x}, upsert=True )


{'/IRS990/ActivityOrMissionDesc': 'SEE BELOW',
 '/IRS990/AllOtherContributionsAmt': '4833',
 '/IRS990/AllOtherExpensesGrp/TotalAmt': '290',
 '/IRS990/BooksInCareOfDetail/PersonNm': 'THOMAS TOBIN',
 '/IRS990/BooksInCareOfDetail/PhoneNum': '7817629724',
 '/IRS990/BooksInCareOfDetail/USAddress/AddressLine1': '152 WINSLOW AVENIUE',
 '/IRS990/BooksInCareOfDetail/USAddress/City': 'NORWOOD',
 '/IRS990/BooksInCareOfDetail/USAddress/State': 'MA',
 '/IRS990/BooksInCareOfDetail/USAddress/ZIPCode': '02062',
 '/IRS990/CYBenefitsPaidToMembersAmt': '0',
 '/IRS990/CYContributionsGrantsAmt': '4833',
 '/IRS990/CYGrantsAndSimilarPaidAmt': '0',
 '/IRS990/CYInvestmentIncomeAmt': '0',
 '/IRS990/CYOtherExpensesAmt': '87541',
 '/IRS990/CYOtherRevenueAmt': '0',
 '/IRS990/CYProgramServiceRevenueAmt': '50710',
 '/IRS990/CYRevenuesLessExpensesAmt': '-31998',
 '/IRS990/CYSalariesCompEmpBnftPaidAmt': '0',
 '/IRS990/CYTotalExpensesAmt': '87541',
 '/IRS990/CYTotalFundraisingExpenseAmt': '0',
 '/IRS990/CYTotalProfFndr

{'/IRS990/ActivityOrMissionDesc': 'SEE BELOW',
 '/IRS990/AllOtherContributionsAmt': '4833',
 '/IRS990/AllOtherExpensesGrp/TotalAmt': '290',
 '/IRS990/BooksInCareOfDetail/PersonNm': 'THOMAS TOBIN',
 '/IRS990/BooksInCareOfDetail/PhoneNum': '7817629724',
 '/IRS990/BooksInCareOfDetail/USAddress/AddressLine1': '152 WINSLOW AVENIUE',
 '/IRS990/BooksInCareOfDetail/USAddress/City': 'NORWOOD',
 '/IRS990/BooksInCareOfDetail/USAddress/State': 'MA',
 '/IRS990/BooksInCareOfDetail/USAddress/ZIPCode': '02062',
 '/IRS990/CYBenefitsPaidToMembersAmt': '0',
 '/IRS990/CYContributionsGrantsAmt': '4833',
 '/IRS990/CYGrantsAndSimilarPaidAmt': '0',
 '/IRS990/CYInvestmentIncomeAmt': '0',
 '/IRS990/CYOtherExpensesAmt': '87541',
 '/IRS990/CYOtherRevenueAmt': '0',
 '/IRS990/CYProgramServiceRevenueAmt': '50710',
 '/IRS990/CYRevenuesLessExpensesAmt': '-31998',
 '/IRS990/CYSalariesCompEmpBnftPaidAmt': '0',
 '/IRS990/CYTotalExpensesAmt': '87541',
 '/IRS990/CYTotalFundraisingExpenseAmt': '0',
 '/IRS990/CYTotalProfFndr

{'/IRS990/ActivityOrMissionDesc': 'SEE BELOW',
 '/IRS990/AllOtherContributionsAmt': '4833',
 '/IRS990/AllOtherExpensesGrp/TotalAmt': '290',
 '/IRS990/BooksInCareOfDetail/PersonNm': 'THOMAS TOBIN',
 '/IRS990/BooksInCareOfDetail/PhoneNum': '7817629724',
 '/IRS990/BooksInCareOfDetail/USAddress/AddressLine1': '152 WINSLOW AVENIUE',
 '/IRS990/BooksInCareOfDetail/USAddress/City': 'NORWOOD',
 '/IRS990/BooksInCareOfDetail/USAddress/State': 'MA',
 '/IRS990/BooksInCareOfDetail/USAddress/ZIPCode': '02062',
 '/IRS990/CYBenefitsPaidToMembersAmt': '0',
 '/IRS990/CYContributionsGrantsAmt': '4833',
 '/IRS990/CYGrantsAndSimilarPaidAmt': '0',
 '/IRS990/CYInvestmentIncomeAmt': '0',
 '/IRS990/CYOtherExpensesAmt': '87541',
 '/IRS990/CYOtherRevenueAmt': '0',
 '/IRS990/CYProgramServiceRevenueAmt': '50710',
 '/IRS990/CYRevenuesLessExpensesAmt': '-31998',
 '/IRS990/CYSalariesCompEmpBnftPaidAmt': '0',
 '/IRS990/CYTotalExpensesAmt': '87541',
 '/IRS990/CYTotalFundraisingExpenseAmt': '0',
 '/IRS990/CYTotalProfFndr

{'/IRS990/ActivityOrMissionDesc': 'SEE BELOW',
 '/IRS990/AllOtherContributionsAmt': '4833',
 '/IRS990/AllOtherExpensesGrp/TotalAmt': '290',
 '/IRS990/BooksInCareOfDetail/PersonNm': 'THOMAS TOBIN',
 '/IRS990/BooksInCareOfDetail/PhoneNum': '7817629724',
 '/IRS990/BooksInCareOfDetail/USAddress/AddressLine1': '152 WINSLOW AVENIUE',
 '/IRS990/BooksInCareOfDetail/USAddress/City': 'NORWOOD',
 '/IRS990/BooksInCareOfDetail/USAddress/State': 'MA',
 '/IRS990/BooksInCareOfDetail/USAddress/ZIPCode': '02062',
 '/IRS990/CYBenefitsPaidToMembersAmt': '0',
 '/IRS990/CYContributionsGrantsAmt': '4833',
 '/IRS990/CYGrantsAndSimilarPaidAmt': '0',
 '/IRS990/CYInvestmentIncomeAmt': '0',
 '/IRS990/CYOtherExpensesAmt': '87541',
 '/IRS990/CYOtherRevenueAmt': '0',
 '/IRS990/CYProgramServiceRevenueAmt': '50710',
 '/IRS990/CYRevenuesLessExpensesAmt': '-31998',
 '/IRS990/CYSalariesCompEmpBnftPaidAmt': '0',
 '/IRS990/CYTotalExpensesAmt': '87541',
 '/IRS990/CYTotalFundraisingExpenseAmt': '0',
 '/IRS990/CYTotalProfFndr

InvalidDocument: Cannot encode object: None

...

InvalidDocument: Cannot encode object: None

InvalidDocument: Cannot encode object: None

InvalidDocument: Cannot encode object: None

InvalidDocument: Cannot encode object: None

InvalidDocument: Cannot encode object: None